In [3]:
#!/usr/bin/env Rscript
# ==========================
# Part A — CFA comparisons
# ==========================

quiet_install <- function(pkgs){
  need <- pkgs[!pkgs %in% rownames(installed.packages())]
  if (length(need)) install.packages(need, quiet = TRUE, dependencies = TRUE)
}
quiet_install(c("lavaan","dplyr","stringr","readr","knitr"))

suppressPackageStartupMessages({
  library(lavaan); library(dplyr); library(stringr); library(readr); library(knitr)
})

set.seed(1234)

# ---------- Config ----------
DATA_FILE <- "local_cdr_keep_after_attn.csv"
OUT_DIR   <- "out_sem_cdr"
dir.create(OUT_DIR, showWarnings = FALSE, recursive = TRUE)

# ---------- Helpers ----------
convert_likert <- function(x){
  x <- trimws(as.character(x))
  lead <- stringr::str_match(x, "^([1-7])")[,2]
  out  <- suppressWarnings(as.numeric(ifelse(!is.na(lead), lead, x)))
  out[is.na(out)] <- NA_real_; out
}
as_ord_1to7 <- function(x, min_count = 5){
  v <- convert_likert(x); if (all(is.na(v))) return(NULL)
  v <- pmin(pmax(round(v), 1), 7)
  tab <- table(v, useNA = "no")
  if ("1" %in% names(tab) && tab["1"] < min_count) v[v==1] <- 2
  tab <- table(v, useNA = "no")
  if ("7" %in% names(tab) && tab["7"] < min_count) v[v==7] <- 6
  if (length(unique(na.omit(v))) < 2) return(NULL)
  ordered(v, levels = sort(unique(v)))
}

# Robust fit_row (tolerates length-0 returns from fitMeasures)
fit_row <- function(m, label, n_fallback = NA_integer_) {
  keys <- c("chisq","df","pvalue","cfi","tli","rmsea",
            "rmsea.ci.lower","rmsea.ci.upper","srmr","nobs")
  get_safe <- function(k) {
    val <- tryCatch(fitMeasures(m, k), error = function(e) numeric(0))
    if (length(val) == 0) NA_real_ else as.numeric(val)
  }
  vals <- setNames(lapply(keys, get_safe), keys)
  if (length(vals$nobs) == 0 || is.na(vals$nobs)) {
    vals$nobs <- tryCatch(nobs(m), error = function(e) n_fallback)
  }
  data.frame(
    Model = label,
    CHISQ = vals$chisq, DF = vals$df, PVALUE = vals$pvalue,
    CFI = vals$cfi, TLI = vals$tli, SRMR = vals$srmr,
    RMSEA = vals$rmsea, RMSEA.CI.LOWER = vals$`rmsea.ci.lower`,
    RMSEA.CI.UPPER = vals$`rmsea.ci.upper`, NOBS = vals$nobs,
    check.names = FALSE
  )
}
save_tbl <- function(df, stem){
  csv <- file.path(OUT_DIR, paste0(stem, ".csv"))
  md  <- file.path(OUT_DIR, paste0(stem, ".md"))
  readr::write_csv(df, csv)
  writeLines(paste(capture.output(knitr::kable(df, format="markdown")), collapse="\n"), md, useBytes=TRUE)
  invisible(list(csv=csv, md=md))
}
fit_with_rescue <- function(model_txt, data, ord_cols, label="CFA"){
  ladder <- expand.grid(est=c("WLSMV","DWLS"), st=c(FALSE, TRUE),
                        KEEP.OUT.ATTRS=FALSE, stringsAsFactors=FALSE)
  for (k in seq_len(nrow(ladder))) {
    est <- ladder$est[k]; st <- ladder$st[k]
    cat(sprintf("\nTrying: est=%s, par=theta, start_simple=%s\n", est, st))
    fit <- try(lavaan::cfa(
      model            = model_txt,
      data             = data,
      ordered          = ord_cols,
      estimator        = est,
      parameterization = "theta",
      std.lv           = FALSE,         # marker-ID
      missing          = "listwise",
      start            = if (st) "simple" else NULL,
      control          = list(iter.max = 10000, rel.tol = 1e-6)
    ), silent = TRUE)
    ok <- !inherits(fit,"try-error") && isTRUE(tryCatch(lavInspect(fit,"converged"), error=function(e) FALSE))
    if (ok) { cat("Converged: TRUE\n"); return(fit) } else { cat("Converged: FALSE\n") }
  }
  stop(sprintf("All rescue attempts failed (%s).", label))
}

# ---------- Read & assemble data ----------
stopifnot(file.exists(DATA_FILE))
cat("Reading:", normalizePath(DATA_FILE), "\n")
raw <- readr::read_csv(DATA_FILE, show_col_types=FALSE) |> as.data.frame()

cdr_cols <- grep("^(ENV|SOC|GOV)_\\d{2}$", names(raw), value=TRUE)
tolerant_map <- function(){
  out <- vector("list", 20); names(out) <- as.character(1:20)
  for (i in 1:20) { pat <- paste0("CDR attributions _", i, "$")
    hit <- grep(pat, names(raw), value=TRUE); if (length(hit)) out[[as.character(i)]] <- hit[1] }
  out
}
if (length(cdr_cols) < 6) {
  m <- tolerant_map()
  if (!length(Filter(Negate(is.null), m))) stop("Provide ENV_01..GOV_20 or 'CDR attributions _1..20'.")
  tag_from_num <- function(i){ if (i<=3) sprintf("ENV_%02d", i) else if (i<=12) sprintf("SOC_%02d", i) else sprintf("GOV_%02d", i) }
  X <- data.frame(row_id=seq_len(nrow(raw)))
  for (i in 1:20){ src <- m[[as.character(i)]]; if (!is.null(src)) X[[tag_from_num(i)]] <- raw[[src]] }
  X$row_id <- NULL
} else {
  X <- raw[, cdr_cols, drop=FALSE]
}
XO <- lapply(X, as_ord_1to7)
keep <- !vapply(XO, is.null, logical(1))
if (!all(keep)) message("Dropped ESG items with <2 categories: ", paste(names(XO)[!keep], collapse=", "))
DF <- as.data.frame(XO[keep], check.names=FALSE)
ordered_cols <- names(DF)[vapply(DF, is.ordered, logical(1))]

ITEMS <- list(
  Environmental = grep("^ENV_\\d{2}$", names(DF), value=TRUE),
  Social        = grep("^SOC_\\d{2}$", names(DF), value=TRUE),
  Governance    = grep("^GOV_\\d{2}$", names(DF), value=TRUE)
)

# Ambiguous per EFA (loaded with Governance)
AMBIG_SOC <- intersect(c("SOC_05","SOC_09","SOC_11","SOC_12"), ITEMS$Social)

# ---------- CFA model builders (first-order, correlated factors) ----------
cfa_text_from_lists <- function(L){
  stopifnot(length(L$Environmental)>=2, length(L$Social)>=2, length(L$Governance)>=2)
  paste0(
    "Environmental =~ ", paste(L$Environmental, collapse=" + "), "\n",
    "Social        =~ ", paste(L$Social,        collapse=" + "), "\n",
    "Governance    =~ ", paste(L$Governance,    collapse=" + "), "\n",
    "Environmental ~~ Social\n",
    "Environmental ~~ Governance\n",
    "Social ~~ Governance\n"
  )
}

LIST_theory   <- ITEMS
LIST_EFAshift <- within(as.list(ITEMS), {
  Social     <- setdiff(Social, AMBIG_SOC)
  Governance <- sort(unique(c(Governance, AMBIG_SOC)))
})
LIST_dropAmbig <- within(as.list(ITEMS), {
  Social     <- setdiff(Social, AMBIG_SOC)
  Governance <- Governance  # unchanged
})

TXT <- list(
  CFA_3F_theory    = cfa_text_from_lists(LIST_theory),
  CFA_3F_EFAshift  = cfa_text_from_lists(LIST_EFAshift),
  CFA_3F_dropAmbig = cfa_text_from_lists(LIST_dropAmbig)
)

# ---------- Fit ----------
fits <- list()
for (nm in names(TXT)) {
  cat(sprintf("\n[ CFA ] Fitting %s ...\n", nm))
  fits[[nm]] <- fit_with_rescue(TXT[[nm]], DF, ordered_cols, label=nm)
}

# ---------- Output: fit table ----------
fit_tab <- dplyr::bind_rows(
  fit_row(fits$CFA_3F_theory,    "CFA_3F_theory",    nrow(DF)),
  fit_row(fits$CFA_3F_EFAshift,  "CFA_3F_EFAshift",  nrow(DF)),
  fit_row(fits$CFA_3F_dropAmbig, "CFA_3F_dropAmbig", nrow(DF))
) %>% dplyr::mutate(dplyr::across(where(is.numeric), ~as.numeric(.)))

cat("\n# CFA model comparison\n")
print(knitr::kable(fit_tab, format="markdown"))
save_tbl(fit_tab, "cfa_fit_comparison")

# ---------- Output: standardized loadings ----------
std_loads <- lapply(names(fits), function(nm){
  SS <- standardizedSolution(fits[[nm]])
  SS %>% dplyr::filter(op=="=~", lhs %in% c("Environmental","Social","Governance")) %>%
    dplyr::select(Model = lhs, Factor = lhs, Item = rhs, Std_Loading = est.std) %>%
    dplyr::mutate(Spec = nm)
}) |> dplyr::bind_rows()

save_tbl(std_loads, "cfa_standardized_loadings")

cat("\n✅ CFA Part A complete. Outputs in:", normalizePath(OUT_DIR), "\n")


Reading: /home/akroon/corporate-digital-responsibility/local_cdr_keep_after_attn.csv 

[ CFA ] Fitting CFA_3F_theory ...

Trying: est=WLSMV, par=theta, start_simple=FALSE
Converged: TRUE

[ CFA ] Fitting CFA_3F_EFAshift ...

Trying: est=WLSMV, par=theta, start_simple=FALSE
Converged: TRUE

[ CFA ] Fitting CFA_3F_dropAmbig ...

Trying: est=WLSMV, par=theta, start_simple=FALSE
Converged: TRUE

# CFA model comparison


|Model            |    CHISQ|  DF| PVALUE|       CFI|       TLI|      SRMR|     RMSEA| RMSEA.CI.LOWER| RMSEA.CI.UPPER| NOBS|
|:----------------|--------:|---:|------:|---------:|---------:|---------:|---------:|--------------:|--------------:|----:|
|CFA_3F_theory    | 736.6969| 167|      0| 0.9990836| 0.9989574| 0.0230607| 0.0393958|      0.0365128|      0.0423256| 2199|
|CFA_3F_EFAshift  | 400.4132| 167|      0| 0.9996245| 0.9995728| 0.0164251| 0.0252168|      0.0220621|      0.0283909| 2199|
|CFA_3F_dropAmbig | 252.3519| 101|      0| 0.9995831| 0.9995046| 0.0163655| 0.02

In [8]:
# --- Safety check: ensure model exists ---
fit <- ensure_fit("CFA_3F_dropAmbig")

# ==========================================
# 1) UNSTANDARDIZED LOADINGS
# ==========================================
cat("\n==============================\n")
cat("TABLE 1: UNSTANDARDIZED FACTOR LOADINGS\n")
cat("(These are raw loadings, marker items = 1.0 by identification constraint)\n")
cat("==============================\n\n")

unstd <- parameterEstimates(fit) %>%
  dplyr::filter(op == "=~", lhs %in% c("Environmental","Social","Governance")) %>%
  dplyr::select(Factor = lhs, Item = rhs, Unstd_Loading = est, SE = se, Z = z, PValue = pvalue) %>%
  dplyr::arrange(Factor, Item)

print(knitr::kable(unstd, format="markdown"))
save_tbl(unstd, "CFA_3F_dropAmbig_unstandardized_loadings")


# ==========================================
# 2) STANDARDIZED LOADINGS
# ==========================================
cat("\n==============================\n")
cat("TABLE 2: STANDARDIZED FACTOR LOADINGS\n")
cat("(These show strength of relationship in SD units, easier to compare across items)\n")
cat("==============================\n\n")

std <- standardizedSolution(fit, se = TRUE) %>%
  dplyr::filter(op == "=~", lhs %in% c("Environmental","Social","Governance")) %>%
  dplyr::select(Factor = lhs, Item = rhs, Std_Loading = est.std,
                SE_Std = se, Z_Std = z, PValue_Std = pvalue) %>%
  dplyr::arrange(Factor, Item)

print(knitr::kable(std, format="markdown"))
save_tbl(std, "CFA_3F_dropAmbig_standardized_loadings")


# ==========================================
# 3) COMBINED TABLE (UNSTD + STD)
# ==========================================
cat("\n==============================\n")
cat("TABLE 3: COMBINED LOADINGS (Unstandardized + Standardized)\n")
cat("(This is best for export or appendix — contains both metrics side by side)\n")
cat("==============================\n\n")

combined <- dplyr::left_join(
  unstd,
  dplyr::select(std, Factor, Item, Std_Loading, SE_Std, Z_Std, PValue_Std),
  by = c("Factor","Item")
) %>% dplyr::arrange(Factor, Item)

print(knitr::kable(combined, format="markdown"))
save_tbl(combined, "CFA_3F_dropAmbig_all_loadings")

cat("\n✅ Saved all outputs to folder:", normalizePath(OUT_DIR), "\n")



TABLE 1: UNSTANDARDIZED FACTOR LOADINGS
(These are raw loadings, marker items = 1.0 by identification constraint)



|Factor        |Item   | Unstd_Loading|        SE|        Z| PValue|
|:-------------|:------|-------------:|---------:|--------:|------:|
|Environmental |ENV_01 |     1.0000000| 0.0000000|       NA|     NA|
|Environmental |ENV_02 |     1.1396157| 0.0401264| 28.40062|      0|
|Environmental |ENV_03 |     1.3198487| 0.0517104| 25.52388|      0|
|Governance    |GOV_13 |     1.0000000| 0.0000000|       NA|     NA|
|Governance    |GOV_14 |     0.9826403| 0.0279849| 35.11320|      0|
|Governance    |GOV_15 |     0.9876271| 0.0271719| 36.34741|      0|
|Governance    |GOV_16 |     0.9635220| 0.0259784| 37.08931|      0|
|Governance    |GOV_17 |     0.9816199| 0.0280972| 34.93658|      0|
|Governance    |GOV_18 |     1.0278625| 0.0286295| 35.90226|      0|
|Governance    |GOV_19 |     0.8753551| 0.0254927| 34.33746|      0|
|Governance    |GOV_20 |     0.9936937| 0.0300785| 33.

In [7]:
# Quick CR and AVE from standardized loadings
library(dplyr)

std <- standardizedSolution(fits$CFA_3F_dropAmbig) %>%
  filter(op == "=~", lhs %in% c("Environmental","Social","Governance")) %>%
  transmute(Factor = lhs, Item = rhs, lambda = est.std)

summ <- std %>%
  group_by(Factor) %>%
  summarise(
    k = n(),
    mean_loading = mean(lambda),
    CR  = (sum(lambda)^2) / ((sum(lambda)^2) + sum(1 - lambda^2)),
    AVE = mean(lambda^2),
    .groups = "drop"
  )

knitr::kable(summ, digits = 3)




|Factor        |  k| mean_loading|    CR|   AVE|
|:-------------|--:|------------:|-----:|-----:|
|Environmental |  3|        0.826| 0.866| 0.682|
|Governance    |  8|        0.859| 0.957| 0.738|
|Social        |  5|        0.756| 0.870| 0.573|

In [9]:
# ---- Helper to print and save tables for any fitted model ----
print_and_save_loadings <- function(model_name, fit_obj) {
  cat("\n\n========================================")
  cat(paste0("\nPROCESSING MODEL: ", model_name))
  cat("\n========================================\n")

  # 1) Unstandardized
  cat("\n------------------------------\n")
  cat("UNSTANDARDIZED FACTOR LOADINGS\n")
  cat("(Raw loadings, marker items fixed at 1.0)\n")
  cat("------------------------------\n\n")
  unstd <- parameterEstimates(fit_obj) %>%
    dplyr::filter(op == "=~", lhs %in% c("Environmental","Social","Governance")) %>%
    dplyr::select(Factor = lhs, Item = rhs, Unstd_Loading = est, SE = se, Z = z, PValue = pvalue) %>%
    dplyr::arrange(Factor, Item)
  print(knitr::kable(unstd, format = "markdown"))
  save_tbl(unstd, paste0(model_name, "_unstandardized_loadings"))

  # 2) Standardized
  cat("\n------------------------------\n")
  cat("STANDARDIZED FACTOR LOADINGS\n")
  cat("(Strength of relationship in SD units)\n")
  cat("------------------------------\n\n")
  std <- standardizedSolution(fit_obj, se = TRUE) %>%
    dplyr::filter(op == "=~", lhs %in% c("Environmental","Social","Governance")) %>%
    dplyr::select(Factor = lhs, Item = rhs, Std_Loading = est.std,
                  SE_Std = se, Z_Std = z, PValue_Std = pvalue) %>%
    dplyr::arrange(Factor, Item)
  print(knitr::kable(std, format = "markdown"))
  save_tbl(std, paste0(model_name, "_standardized_loadings"))

  # 3) Combined
  cat("\n------------------------------\n")
  cat("COMBINED TABLE (Unstandardized + Standardized)\n")
  cat("(Best for full appendix/export)\n")
  cat("------------------------------\n\n")
  combined <- dplyr::left_join(
    unstd,
    dplyr::select(std, Factor, Item, Std_Loading, SE_Std, Z_Std, PValue_Std),
    by = c("Factor", "Item")
  ) %>%
    dplyr::arrange(Factor, Item)
  print(knitr::kable(combined, format = "markdown"))
  save_tbl(combined, paste0(model_name, "_all_loadings"))

  cat("\n✅ Finished:", model_name, "— tables saved.\n")
}

# ---- Loop over all models ----
model_names <- c("CFA_3F_theory", "CFA_3F_EFAshift", "CFA_3F_dropAmbig")

for (mn in model_names) {
  fit_obj <- ensure_fit(mn)  # ensures each model is available/refitted if needed
  print_and_save_loadings(mn, fit_obj)
}

cat("\n🎉 All model loading tables exported to:", normalizePath(OUT_DIR), "\n")


Refitting CFA_3F_theory ...




[CFA_3F_theory] Trying: est=WLSMV, theta, start_simple=FALSE
Converged: TRUE


PROCESSING MODEL: CFA_3F_theory

------------------------------
UNSTANDARDIZED FACTOR LOADINGS
(Raw loadings, marker items fixed at 1.0)
------------------------------



|Factor        |Item   | Unstd_Loading|        SE|        Z| PValue|
|:-------------|:------|-------------:|---------:|--------:|------:|
|Environmental |ENV_01 |     1.0000000| 0.0000000|       NA|     NA|
|Environmental |ENV_02 |     1.1438796| 0.0402201| 28.44050|      0|
|Environmental |ENV_03 |     1.3251696| 0.0521305| 25.42024|      0|
|Governance    |GOV_13 |     1.0000000| 0.0000000|       NA|     NA|
|Governance    |GOV_14 |     0.9793144| 0.0275548| 35.54060|      0|
|Governance    |GOV_15 |     0.9790648| 0.0263986| 37.08776|      0|
|Governance    |GOV_16 |     0.9508223| 0.0254336| 37.38455|      0|
|Governance    |GOV_17 |     0.9873410| 0.0281551| 35.06793|      0|
|Governance    |GOV_18 |     1.0137195| 0.0278570| 36.39010

Refitting CFA_3F_EFAshift ...




[CFA_3F_EFAshift] Trying: est=WLSMV, theta, start_simple=FALSE
Converged: TRUE


PROCESSING MODEL: CFA_3F_EFAshift

------------------------------
UNSTANDARDIZED FACTOR LOADINGS
(Raw loadings, marker items fixed at 1.0)
------------------------------



|Factor        |Item   | Unstd_Loading|        SE|        Z| PValue|
|:-------------|:------|-------------:|---------:|--------:|------:|
|Environmental |ENV_01 |     1.0000000| 0.0000000|       NA|     NA|
|Environmental |ENV_02 |     1.1438148| 0.0401848| 28.46387|      0|
|Environmental |ENV_03 |     1.3270640| 0.0522090| 25.41829|      0|
|Governance    |GOV_13 |     1.0000000| 0.0000000|       NA|     NA|
|Governance    |GOV_14 |     0.9798742| 0.0269780| 36.32123|      0|
|Governance    |GOV_15 |     0.9798808| 0.0258598| 37.89199|      0|
|Governance    |GOV_16 |     0.9531008| 0.0250018| 38.12134|      0|
|Governance    |GOV_17 |     0.9877657| 0.0275549| 35.84714|      0|
|Governance    |GOV_18 |     1.0145365| 0.0272904| 37.1

In [10]:
# Packages we need
quiet_install(c("psych"))
suppressPackageStartupMessages(library(psych))

# -------------------------------
# 1) Get final (dropAmbig) items
# -------------------------------
# Uses the exact sets your script built
final_lists <- within(as.list(ITEMS), {
  Social <- setdiff(Social, AMBIG_SOC)  # drop ambiguous Social items
  # Environmental and Governance unchanged
})
stopifnot(length(final_lists$Environmental)>=2,
          length(final_lists$Social)>=2,
          length(final_lists$Governance)>=2)

domains <- c("Environmental","Social","Governance")

# Helper to coerce ordered factors (1-7) to numeric 1..7 safely
to_num_df <- function(cols) {
  out <- as.data.frame(lapply(cols, function(v) {
    x <- if (is.ordered(v)) as.numeric(v) else suppressWarnings(as.numeric(v))
    return(x)
  }), check.names = FALSE)
  out
}

# ---------------------------------------------
# 2) Cronbach's alpha (standard & ordinal alpha)
# ---------------------------------------------
alpha_rows <- lapply(domains, function(dom){
  items <- final_lists[[dom]]
  dat_num <- to_num_df(DF[, items, drop = FALSE])

  # Standard alpha (treat Likert as continuous)
  a_std <- tryCatch({
    psych::alpha(dat_num)$total$std.alpha
  }, error = function(e) NA_real_)

  # Ordinal alpha (based on polychoric correlations)
  a_ord <- tryCatch({
    Rpoly <- psych::polychoric(dat_num)$rho
    psych::alpha(Rpoly, n.obs = nrow(dat_num))$total$std.alpha
  }, error = function(e) NA_real_)

  data.frame(
    Domain = dom,
    k_items = ncol(dat_num),
    Alpha_Std = a_std,
    Alpha_Ordinal = a_ord,
    stringsAsFactors = FALSE
  )
})
alpha_tab <- do.call(rbind, alpha_rows)

cat("\n==============================\n")
cat("RELIABILITY: Cronbach's alpha per concept\n")
cat("(Std = treating Likert as continuous; Ordinal = polychoric-based)\n")
cat("==============================\n\n")
print(knitr::kable(alpha_tab, format="markdown", digits = 3))
save_tbl(alpha_tab, "CFA_3F_dropAmbig_alpha_by_domain")

# ---------------------------------------------
# 3) Scale means and SDs (unit-weighted averages)
# ---------------------------------------------
# Rule: require at least 50% of items present to compute a person's scale score
min_prop <- 0.5

scale_scores <- lapply(domains, function(dom){
  items <- final_lists[[dom]]
  dat_num <- to_num_df(DF[, items, drop = FALSE])

  # person-level mean with missing handling
  n_items <- ncol(dat_num)
  keep_counts <- rowSums(!is.na(dat_num))
  scores <- ifelse(keep_counts / n_items >= min_prop, rowMeans(dat_num, na.rm = TRUE), NA_real_)

  data.frame(
    Domain = dom,
    Mean = mean(scores, na.rm = TRUE),
    SD   = sd(scores, na.rm = TRUE),
    N_Used = sum(!is.na(scores)),
    stringsAsFactors = FALSE
  )
})
desc_tab <- do.call(rbind, scale_scores)

cat("\n==============================\n")
cat("DESCRIPTIVES: Scale means and SDs per concept\n")
cat("(Scores = unit-weighted average of items; require ≥50% items present)\n")
cat("==============================\n\n")
print(knitr::kable(desc_tab, format="markdown", digits = 3))
save_tbl(desc_tab, "CFA_3F_dropAmbig_scale_means_sds")

cat("\n✅ Saved reliability and descriptives to:", normalizePath(OUT_DIR), "\n")

# ---------------------------------------------
# 4) (Optional) item-level descriptives
# ---------------------------------------------
# If you want per-item M/SD as well, uncomment below:
# item_desc <- do.call(rbind, lapply(domains, function(dom){
#   items <- final_lists[[dom]]
#   dat_num <- to_num_df(DF[, items, drop = FALSE])
#   data.frame(
#     Domain = dom,
#     Item = colnames(dat_num),
#     Mean = sapply(dat_num, function(x) mean(x, na.rm=TRUE)),
#     SD   = sapply(dat_num, function(x) sd(x, na.rm=TRUE)),
#     stringsAsFactors = FALSE
#   )
# }))
# print(knitr::kable(item_desc, format="markdown", digits = 3))
# save_tbl(item_desc, "CFA_3F_dropAmbig_item_means_sds")



RELIABILITY: Cronbach's alpha per concept
(Std = treating Likert as continuous; Ordinal = polychoric-based)



|Domain        | k_items| Alpha_Std| Alpha_Ordinal|
|:-------------|-------:|---------:|-------------:|
|Environmental |       3|     0.831|         0.863|
|Social        |       5|     0.840|         0.870|
|Governance    |       8|     0.943|         0.957|

DESCRIPTIVES: Scale means and SDs per concept
(Scores = unit-weighted average of items; require ≥50% items present)



|Domain        |  Mean|    SD| N_Used|
|:-------------|-----:|-----:|------:|
|Environmental | 5.332| 1.239|   2199|
|Social        | 5.227| 1.115|   2199|
|Governance    | 5.620| 1.165|   2199|

✅ Saved reliability and descriptives to: /home/akroon/corporate-digital-responsibility/out_sem_cdr 
